# Advertisement Success Dataset

In [ ]:
import numpy as np
import pandas as pd
import os 

В качестве перового датасета я выбрала датасет "Advertisement Success Dataset". Он включает разные характеристики рекламных кампаний, а именно:

    1. ее идентификатор (id)
    2. параметры, описывающие потенциальный сегмент заинтересованных в ней лиц:
        * гендер (targeted_sex: Male/ Female)
        * семейное положение (relationship_status: Married/ Never-Married/ Divorced/ Separated/ Widowed)
    3. сфера (industry: Auto/ Pharma/ Entertainment/ Political/ ClassAction/ Other)
    4. жанр (genre: Comedy/ Drama/ Infomercial/ Direct/ Other)
    3. средняя продолжительность(кол-во часов в неделе)
    4. эфирное время (airtime: Morning/ Daytime/ Primetime)
    5. страна (airlocation: more than 40 countries)
    6. рейтинги (ratings: 0-1)
    7. дороговизна рекламы (expensive: Low/ High/ Medium)
    8. гарантия возврата средств (money_back_garantee: Yes/ No)
    9. чистая прибыль (netgain: true/ false)

Об успехе рекламной кампании можно судить исходя из признака "чистая прибыль". Именно этот признак я взяла в качестве целевого. Таким образом, этот датасет будем рассматривать в качетсве обучающего для будущей модели.
Модель будет классифицировать рекламу на успешную (netgain = true) и неуспешную (netgain = false) по выбранному целевому признаку.

Рассмотрим данные поближе.

In [ ]:
ad_suc_file_path = '../input/advertsuccess/Train.csv'
data = pd.read_csv(ad_suc_file_path)
data.head()

In [ ]:
data.columns

# Описание данных

In [ ]:
data.describe(include='all')

В данных нет пропусков, т.к. признак count имеет одинаковое значение для всех колонок (характеристик). Часто фигурирует значение NaN из-за того, что для числовых признаков вычисляются самое распространенное значение (top), его частота (freq) и число уникальных признаков (unique) или же для объектных признаков (строки/ временные метки) вычисляются среднее значение (mean), среднеквадратичное значение (std), минимум (min), макисмум (max) и процентили (25, 50, 75%).

Попробуем это исправить.

In [ ]:
data.describe()

In [ ]:
data.describe(include=['O'])

Найдем рекламу комедийного жанра, которая принесла прибыль и считается успешной.

In [ ]:
data_x = data[['genre', 'netgain']]
data_x.loc[(data.netgain == True)]
#data[['genre','netgain']]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
data[['id', 'average_runtime(minutes_per_week)', 'ratings']].hist(bins=50,figsize=(20,15))
#imp_data.hist(bins=50,figsize=(20,15))
#plt.show()

Попробую использовать в качестве модели DecisionTreeRegressor из sklearn.tree.
Для этого выберем числовые признаки, которые имеются. Целевым признаком будет 'ratings'. 

In [ ]:
data['ratings'].min(),data['ratings'].max()

In [ ]:
features1 = ['average_runtime(minutes_per_week)']
X1 = data[features1]
#y = pd.get_dummies(data.netgain)
y1 = data.ratings

In [ ]:
from sklearn.model_selection import train_test_split
train_X1, val_X1, train_y1, val_y1 = train_test_split(X1, y1, random_state=1)
print(len(train_X1), "train +", len(val_X1), "test")
train_X1.head()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

as_model1 = DecisionTreeRegressor(random_state=1)
as_model1.fit(train_X1, train_y1)
val_predictions1 = as_model1.predict(val_X1)
as_mae1 = mean_absolute_error(val_predictions1, val_y1)

print(val_predictions1[0:5])
print(val_y1.head().values)
as_mae1 = mean_absolute_error(val_predictions1, val_y1)
print("Validation MAE when not specifying max_leaf_nodes: ", as_mae1)

Если учесть, что признак 'ratings' принимает значения от 0 до 1, то среднее значение ошибки небольшое.
Чтобы еще уменьшить ее значение, попробуем найти оптимальную высоту дерева решений.

In [ ]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
scores = { leaf_size: get_mae(leaf_size, train_X1, val_X1, train_y1, val_y1) for leaf_size in candidate_max_leaf_nodes}
best_tree_size = min(scores, key=scores.get)
print("Best tree size:", best_tree_size)
print(scores)

Попробуем рассмотреть количество листьев в интервале [2:6].

In [ ]:
candidate_max_leaf_nodes = [2, 3, 4, 5]
scores = { leaf_size: get_mae(leaf_size, train_X1, val_X1, train_y1, val_y1) for leaf_size in candidate_max_leaf_nodes}
best_tree_size = min(scores, key=scores.get)
print("Best tree size:", best_tree_size)
print(scores)

Оптимальное количество листьев равно 5.
Построим модель с учетом количества листьев и сравним среднее значение ошибки с ранее найденным. 

In [ ]:
as_model2 = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=1)
as_model2.fit(train_X1, train_y1)
val_predictions2 = as_model2.predict(val_X1)
as_mae2 = mean_absolute_error(val_predictions2, val_y1)
print(val_predictions2[0:5])
print(val_y1.head().values)
print("Validation MAE for best value of max_leaf_nodes: ", as_mae2)
print("The difference between MAE1 and MAE2: ", as_mae1 - as_mae2)

Небольшое среднее значение ошибки связано с тем, что в тестовых и обучающих данных значения признака 'ratings' сконцентрированы и неравномерны на (0, 1). Они почти идиентичны (это можно видеть из графика).

Тем не менее, проверим обе модели на тестовом датасете и сравним полученные результаты.

In [ ]:
ad_suc_file_path = '../input/advertsuccess/Test.csv'
test_data = pd.read_csv(ad_suc_file_path)
test_data.head()


Посмотрим распределения численных признаков в тестовом датасете.

In [ ]:
test_data.describe()

In [ ]:
%matplotlib inline
test_data[['id', 'average_runtime(minutes_per_week)', 'ratings']].hist(bins=50,figsize=(20,15))

In [ ]:
val_X1 = test_data[features1]
val_predictions1 = as_model1.predict(val_X1)
print(val_predictions1[0:5])

val_predictions2 = as_model2.predict(val_X1)
print(val_predictions2[0:5])
as_mae2 = mean_absolute_error(val_predictions1, val_predictions2)
print("The difference between 2 models on test_data: ", as_mae2)

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ['average_runtime(minutes_per_week)', 'ratings'] 
scatter_matrix(data[attributes], figsize=(12, 8));

Предыдущая модель решала не совсем ту задачу, которая стояла изначально.
Теперь попробуем включить в обучающую выборку не только численные признаки, но и категориальные. Их, как оказалось, намного больше в этом датасете. В качетсве целевого признака возьмем булевский признак "наличие прибыли".

Сначала сделаем ряд преобразований в датасете, чтобы построить графики зависимостей всех признаков. Преобразуем категориальные признаки в числовые.

In [ ]:
num_data = data.copy(deep=True)
num_data['realtionship_status'].value_counts().plot.bar()
    

Из графика видно, что категориальный признак 'relationship_status' принимает 7 значений.

Преобразуем эти значения в численные.

In [ ]:
nv = 0
for value in num_data.realtionship_status.unique():
    num_data.loc[num_data['realtionship_status'] == value, 'realtionship_status'] = nv
    nv +=1
num_data.head()

С остальными категориальными признаками необходимо сделать то же самое.

In [ ]:
num_data['industry'].value_counts().plot.bar()

In [ ]:
#nv = 0
for value in num_data.industry.unique():
    num_data.loc[num_data['industry'] == value, 'industry'] = nv
    nv +=1

In [ ]:
num_data['genre'].value_counts().plot.bar()

In [ ]:
#nv = 0
for value in num_data.genre.unique():
    num_data.loc[num_data['genre'] == value, 'genre'] = nv
    nv +=1

In [ ]:
num_data['targeted_sex'].value_counts().plot.bar()

In [ ]:
num_data.loc[num_data['targeted_sex'] == 'Male', 'targeted_sex'] = 0
num_data.loc[num_data['targeted_sex'] == 'Female', 'targeted_sex'] = 1

In [ ]:
num_data['airtime'].value_counts().plot.bar()

In [ ]:
#nv = 0
for value in num_data.airtime.unique():
    num_data.loc[num_data['airtime'] == value, 'airtime'] = nv
    nv +=1

In [ ]:
num_data['airlocation'].value_counts().plot.bar()

In [ ]:
#nv = 0
for value in num_data.airlocation.unique():
    num_data.loc[num_data['airlocation'] == value, 'airlocation'] = nv
    nv +=1

In [ ]:
num_data['expensive'].value_counts().plot.bar()

In [ ]:
#nv = 0
for value in num_data.expensive.unique():
    num_data.loc[num_data['expensive'] == value, 'expensive'] = nv
    nv +=1

In [ ]:
num_data['money_back_guarantee'].value_counts().plot.bar()

In [ ]:
num_data.loc[num_data['money_back_guarantee'] == 'Yes', 'money_back_guarantee'] = 1
num_data.loc[num_data['money_back_guarantee'] == 'No', 'money_back_guarantee'] = 0
num_data.head()

Наконец, сделаем целевой признак численным.

In [ ]:
num_data['netgain'].value_counts().plot.bar()

In [ ]:
num_data.loc[num_data['netgain'] == True, 'netgain'] = 1
num_data.loc[num_data['netgain'] == False, 'netgain'] = 0
num_data.head()

In [ ]:
y = num_data.netgain
features = ['industry', 'average_runtime(minutes_per_week)', 'airtime', 'airlocation', 'ratings']
X = num_data[features]
#oheX = pd.get_dummies(X)
#ohey = pd.get_dummies(y)

Разобьем данные на тренировочные и тестовые.

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
print(len(train_X), "train +", len(val_X), "test")

Будем в этот раз использовать алгоритм RandomForestRegressor() из sklearn.ensemble.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
as_model = RandomForestRegressor()
#as_model = DecisionTreeRegressor()
as_model.fit(train_X, train_y)
val_predictions = as_model.predict(val_X)

Полученный вектор значений преобразуем в бинарный вид. Если значение >= 0.5, то будем считать его True (1), в обратном случае False (0).

In [ ]:
def back_to_binary(val_predictions):
    for i in range(val_predictions.size):
        if val_predictions[i] >= 0.5:
            val_predictions[i] = 1
        else:
            val_predictions[i] = 0

In [ ]:
back_to_binary(val_predictions)
as_mae = mean_absolute_error(val_predictions, val_y)
print(val_predictions[0:5])
print(val_y.head().values)
print("Validation MAE: ", as_mae)

Проверка модели на всем датасете.

In [ ]:
val_predictions = as_model.predict(X)
back_to_binary(val_predictions)
as_mae = mean_absolute_error(val_predictions, y)
print(val_predictions[0:5])
print(val_y.head().values)
print("Validation MAE: ", as_mae)

In [ ]:
from pandas.plotting import scatter_matrix
attributes = features
scatter_matrix(num_data[attributes], figsize=(30, 20));

In [ ]:
attributes = ['average_runtime(minutes_per_week)', 'netgain']
scatter_matrix(num_data[attributes], figsize=(12, 8));

In [ ]:
attributes = ['ratings', 'netgain']
scatter_matrix(num_data[attributes], figsize=(12, 8));

In [ ]:
attributes = ['industry', 'netgain']
scatter_matrix(num_data[attributes], figsize=(12, 8));

In [ ]:
attributes = ['airtime', 'netgain']
scatter_matrix(num_data[attributes], figsize=(12, 8));

По графикам видно, что почти все категориальные признаки имеют дискретные значения.
Опытным путем я определила оптимальный набор признаков, при которых среднее значение ошибок достигает минимального значения (0.16). Этот набор включает такие признаки, как сфера ('industry'), средняя продолжительность трансляции ('average_runtime(minutes_per_week)'), эфирное время ('airtime'), страна ('airlocation'), рейтинги ('ratings').
Именно на основании этих признаков модель наилучшим образом предсказывает принесет ли реклама прибыль, или же нет.

# Проблемы, с которыми я стокнулась

Я выбрала датасет "Advertisement Success Dataset" и выделила целевой признак — "чистая прибыль", который может принимать значения True и False. Соответсвенно, моей задачей стало определение успеха или неуспеха рекламной компании согласно наличию или отсутствию чистой прибыли. Далее, я рассмотрела все представленные признаки подробнее — убедилась, что в данных нет пропусков и нулевых значений. Также я обнаружила, что большинство признаков, представленных в датасете, являются категориальными, включая выбранный мной целевой признак. Эти признаки принимают дискретные значения.

В связи с этим я немного изменила поставленную задачу и решила обучить модель по численному признаку — "средняя продолжительность трансляции в неделю", и в качетсве целевого признака выбрать признак "рейтинги". В принципе, этот признак тоже влияет на успешность/ провал рекламной компании.
Результат оказался хорошим, среднее значение ошибки составило 2%, но это было связано с тем, что большая часть рейтингов в обучающем и тестовом наборах примерно совпадала (из графика видно, что она колеблется от 0 до 0.3).

Чтобы вернуться к первоначальной задаче, я представила все категориальные признаки в виде числовых. Некоторые из них, включая целевой, были бинарными, большинство принимало счетное число значений, самое большое из которых 42 (страна). Модель строилась по новому датасету. Я построила зависимости всех признаков от целевого и опытным путем определила самые значимые признаки, необходимые для обучения модели. Таким образом, среднее значение ошибки для решение первоначальной задачи = 0.17. Это около 17%.
